In [24]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import scipy.sparse as sp

In [27]:
'''# dummy mtrix
# Define the matrix
matrix = [
    [0, 0.5, 0, 0.1, 0.2, 0.7, 0, 0.3],
    [0.5, 0, 0.7, 0.3, 0.4, 0, 0.4, 0.2],
    [0, 0.7, 0, 0.2, 0, 0.6, 0, 0],
    [0.1, 0.3, 0.2, 0, 0, 0, 0, 0.7],
    [0.2, 0.4, 0, 0, 0, 0.2, 0, 0.5],
    [0.7, 0, 0.6, 0, 0.2, 0, 0.1, 0],
    [0, 0.4, 0, 0, 0, 0.1, 0, 0.3],
    [0.3, 0.2, 0, 0.7, 0.5, 0, 0.3, 0]
]

# Save the matrix into a CSV file
with open('/home/gabridele/Desktop/dummy_matrix.csv', 'w', newline='') as csvfile:
#with open('/Users/gabrieledele/Desktop/dummy_matrix.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for row in matrix:
        writer.writerow(row)'''

In [53]:
connectome_matrix = '/home/gabridele/Desktop/connectome_sub-100206.csv'
path_dummy_mtrix = '/home/gabridele/Desktop/dummy_matrix.csv'
#path_dummy_mtrix = '/Users/gabrieledele/Desktop/dummy_matrix.csv'
path_to_mtrx = '/home/gabridele/Desktop/connectome_sub-100610.csv'
#path_to_mtrx = '/Users/gabrieledele/Desktop/Hub_identification/gabriele_sc_connectomes/connectome_sub-100610.csv'

matrix = np.loadtxt(connectome_matrix, delimiter=',')

#resized_matrix = original_matrix[:100, :100]
#print(resized_matrix.shape)

In [105]:
non_zero_values = matrix[matrix != 0]
if len(non_zero_values) > 0:
    lowest_non_zero_value = np.min(non_zero_values)
    index_of_lowest_non_zero_value = np.unravel_index(np.argmin(matrix), matrix.shape)
    print("Lowest non-zero value:", lowest_non_zero_value)
    print("Index of lowest non-zero value:", index_of_lowest_non_zero_value)
else:
    print("No non-zero values found in the adjacency matrix.")

rows_with_zeros = np.where(np.all(matrix == 0, axis=1))[0]
if len(rows_with_zeros) > 0:
    print("Rows with only zeros:", rows_with_zeros)
else:
    print("No rows with only zeros.")


Lowest non-zero value: 0.00027356
Index of lowest non-zero value: (0, 0)
Rows with only zeros: [715 723]


In [26]:
# Load the connectivity matrix from a CSV file
def load_connectivity_matrix(file_path):
    return np.loadtxt(file_path, delimiter=',')

In [22]:
# Set the activation threshold
## che non ci siano righe/cols con zero
## to each subj their threshold
## far andare simulazione

In [27]:
# Select seed nodes randomly
def select_seed_nodes(num_nodes, num_seeds):
    return np.random.choice(num_nodes, num_seeds, replace=False)

In [100]:
#newest 
def simulation(connectivity_matrix, seed_nodes, max_iterations):
    num_nodes = connectivity_matrix.shape[0]
    #successful_thresholds = []
    all_weighted_degrees = np.sum(connectivity_matrix, axis=0)
    print('num_nodes:', num_nodes)
    print('all_weighted_degrees:', all_weighted_degrees)
    print('seed_nodes:', seed_nodes)

    iteration = 0
    seeds = seed_nodes

    active_nodes = np.zeros(num_nodes, dtype=bool)
    active_nodes[seeds] = True
    activation_array = np.zeros((num_nodes, max_iterations))
    prev_seeds = {}
    new_seeds = []
    while not np.all(active_nodes) and iteration < max_iterations:
        iteration += 1
        print('\n ITERATION:', iteration, '\n')
        if iteration == 1:
            for seed in seeds:
                print('current seed:', seed)
                neighbors = connectivity_matrix[seed]
                print('neighbors:', neighbors)
                for index, neighbor in enumerate(neighbors):
                    if neighbor != 0:
                        #print('index:', index)
                        #print('neighbor:', neighbor)
                        if active_nodes[index]:
                            continue
                        else:
                            edge_weight = neighbor
                            weighted_degree = all_weighted_degrees[index]
                            print('edge_weight:', edge_weight)
                            activation_100 = edge_weight / weighted_degree * 100
                            print('activation_100:', activation_100)
                            activation_array[index, iteration - 1] = activation_100
                            # set theta and save it so that "else" can access it too
                            theta = 0.018
                            print('theta:', theta)
                            if activation_100 > theta:  # Check if activation exceeds threshold
                                print('theta surpassed')
                                active_nodes[index] = True
                                if seed not in prev_seeds:
                                    prev_seeds[seed] = [index]
                                else:
                                    prev_seeds[seed].append(index)
                                print('prev seeds:', prev_seeds)
                                if index not in new_seeds:
                                    new_seeds.append(index)
                                    print('new_seeds:', new_seeds)
                            else:
                                print('theta not surpassed')
        else:
            #print('else')
            next_new_seeds = []
            for new_seed in new_seeds:
                print('current seed:', new_seed)
                neighbors = connectivity_matrix[new_seed]  # New_seed instead of seed
                print('neighbors:', neighbors)
                for index, neighbor in enumerate(neighbors):
                    if neighbor != 0:
                        #print('index:', index)
                        #print('neighbor:', neighbor)
                        if active_nodes[index]:
                            continue
                        else:
                            # Calculate activation from previously infected nodes
                            past_activation = 0
                            for prev_seed in prev_seeds:
                                prev_edge = connectivity_matrix[prev_seed][new_seed]
                                past_activation += prev_edge
                                print('past activ:', past_activation)
                            edge_weight = neighbor
                            weighted_degree = all_weighted_degrees[index]
                            print('edge_weight:', edge_weight)
                            activation_100 = (edge_weight + past_activation) / weighted_degree * 100
                            print('activation_100:', activation_100)
                            activation_array[index, iteration - 1] = activation_100
                            # Set theta and save it so that "else" can access it too
                            theta = 0.018
                            print('theta:', theta)
                            #theta = find_threshold()
                            if activation_100 > theta:  # Check if activation exceeds threshold
                                print('theta surpassed')
                                active_nodes[index] = True
                                if new_seed not in prev_seeds:
                                    prev_seeds[new_seed] = [index]  # Create new entry in prev_seeds
                                else:
                                    prev_seeds[new_seed].append(index)  # Append to existing entry
                                print('prev seeds:', prev_seeds)
                                if index not in next_new_seeds:
                                    next_new_seeds.append(index)
                                    print('next_new_seeds:', next_new_seeds)
                            else:
                                print('theta not surpassed')
            new_seeds = next_new_seeds # Update new seeds for next iteration
    
        print('activation_array:', activation_array)
        print("Active nodes:", active_nodes)
        print("New seeds:", new_seeds)
        num_false_values = np.sum(~active_nodes)
        print("Number of False values in the array:", num_false_values)
        false_indices = np.where(~active_nodes)[0]
        print("Indices of False values in the array:", false_indices)
    print('Complete activation_array:', activation_array)
    return active_nodes

In [124]:
all_weighted_degrees = np.sum(matrix, axis=0)
    
print('all_weighted_degrees:', all_weighted_degrees)


# Filter out indexes with zero weighted degrees
zero_degree_indexes = np.where(all_weighted_degrees == 0)[0]
filtered_connectivity_matrix = matrix[np.where(all_weighted_degrees != 0)]
num_nodes = filtered_connectivity_matrix.shape[0]
print(num_nodes)
if len(zero_degree_indexes) != 0:
    print("There are nodes with zero weighted degrees.")
else:
    num_nodes = filtered_connectivity_matrix.shape[0]
    print('num_nodes:', num_nodes)

# Print non-zero indexes and filtered connectivity matrix
print("Non-zero indexes:", zero_degree_indexes)
print("Filtered connectivity matrix:")
print(filtered_connectivity_matrix)

all_weighted_degrees: [7.66484533 6.90493067 5.53751969 ... 8.26683955 8.66808192 3.00695568]
1052
There are nodes with zero weighted degrees.
Non-zero indexes: [715 723]
Filtered connectivity matrix:
[[0.         0.286721   0.0948226  ... 0.0186514  0.         0.00204638]
 [0.286721   0.         0.0241206  ... 0.0137045  0.         0.00645421]
 [0.0948226  0.0241206  0.         ... 0.0124402  0.         0.00358102]
 ...
 [0.0186514  0.0137045  0.0124402  ... 0.         0.         0.00505051]
 [0.         0.         0.         ... 0.         0.         0.00505051]
 [0.00204638 0.00645421 0.00358102 ... 0.00505051 0.00505051 0.        ]]


In [121]:
connectivity_matrix = np.array([[0, 1, 0],
                                 [1, 0, 0],
                                 [0, 0, 0]])

# Calculate weighted degrees
all_weighted_degrees = np.sum(connectivity_matrix, axis=0)

# Identify rows with zero weighted degrees
zero_degree_indexes = np.where(all_weighted_degrees == 0)[0]

# Filter out rows with zero weighted degrees
filtered_connectivity_matrix = connectivity_matrix[np.where(all_weighted_degrees != 0)]

# Print results
print("Number of nodes with zero weighted degrees:", len(zero_degree_indexes))
print("Filtered connectivity matrix:")
print(filtered_connectivity_matrix)

Number of nodes with zero weighted degrees: 1
Filtered connectivity matrix:
[[0 1 0]
 [1 0 0]]


In [207]:
#experimenting
def simulation2(connectivity_matrixx, num_seeds, max_iterations):
    all_weighted_degreess = np.sum(connectivity_matrixx, axis=0)
    
    print('all_weighted_degrees:', all_weighted_degreess)

    # Filter out indexes with zero weighted degrees
    nonzero_indexes = np.where(all_weighted_degreess != 0)[0]
    connectivity_matrix = connectivity_matrixx[nonzero_indexes][:, nonzero_indexes]
    all_weighted_degrees = np.sum(connectivity_matrix, axis=0)
    num_nodes = connectivity_matrix.shape[0]
    print('num_nodes:', num_nodes)

    seed_nodes = np.random.choice(num_nodes, num_seeds, replace=False)

    iteration = 0
    seeds = seed_nodes
    print('seeds:', seeds)
    active_nodes = np.zeros(num_nodes, dtype=bool)
    print(active_nodes.shape)
    active_nodes[seeds] = True
    activation_array = np.zeros((num_nodes, max_iterations))
    prev_seeds = {}
    new_seeds = []
    activated_nodes = set()
    
    while not np.all(active_nodes) and iteration < max_iterations:
        iteration += 1
        print('\n ITERATION:', iteration, '\n')
        
        if iteration == 1:
            neighbors = connectivity_matrix[seeds]
            activated_indices = np.where(neighbors != 0)
            activated_values = neighbors[activated_indices]
            weighted_degrees = all_weighted_degrees[activated_indices[1]]
            activation_100 = activated_values / weighted_degrees * 100

            for seed, activation in zip(seeds, activation_100):

                theta = 0.001818
                print('theta:', theta)
                if activation > theta:
                    activated_nodes.update(np.where(activation_100 > theta)[0])
                    active_nodes[np.where(activation_100 > theta)[0]] = True
                    prev_seeds[seed] = np.where(activation_100 > theta)[0]
                    new_seeds.extend(np.where(activation_100 > theta)[0])

        else:
            next_new_seeds = []
            for new_seed in new_seeds:
                neighbors = connectivity_matrix[new_seed]  # New_seed instead of seed
                activated_indices = np.where(neighbors != 0)
                activated_values = neighbors[activated_indices]
                
                for index, value in zip(activated_indices[0], activated_values):
                    if not active_nodes[index]:
                        # Calculate activation from previously infected nodes
                        if index in activated_nodes:
                            continue  # Skip calculating activation for nodes activated in previous iterations
                        past_activation = np.sum([connectivity_matrix[prev_seed][new_seed] for prev_seed in prev_seeds])
                        edge_weight = value
                        weighted_degree = all_weighted_degrees[index]
                        activation_100 = (edge_weight + past_activation) / weighted_degree * 100
                        
                        activation_array[index, iteration - 1] = activation_100
                        
                        theta = 0.001818

                        if activation_100 > theta:
                            activated_nodes.add(index)
                            active_nodes[index] = True
                            prev_seeds[new_seed] = [index] if new_seed not in prev_seeds else prev_seeds[new_seed] + [index]
                            next_new_seeds.append(index)
                            
            new_seeds = next_new_seeds  # Update new seeds for next iteration

        print('activation_array:', activation_array)
        print("Active nodes:", active_nodes)
        print("New seeds:", new_seeds)
        num_false_values = np.sum(~active_nodes)
        print("Number of False values in the array:", num_false_values)
        if num_false_values == 0:
            print("All nodes are True. Exiting the loop.")
            break
    print('Complete activation_array:', activation_array)
    return active_nodes

In [ ]:
def find_theta(connectivity_matrix, initial_th):



In [25]:
def find_highest_threshold(connectivity_matrix, max_iterations=10):
    num_nodes = connectivity_matrix.shape[0] # so 8x8 in my dummy case
    successful_thresholds = []  # List to store successful threshold values
    weighted_degree = np.sum(connectivity_matrix, axis=0)
    lowest_weighted_degree = np.min(weighted_degree)
    print("Lowest weighted degree:", lowest_weighted_degree)
    index_lowest_degree = np.argmin(weighted_degree)
    print("with index:", index_lowest_degree)
    seed_nodes = index_lowest_degree

    
    # Initialize active nodes. creates empty array except for
    active_nodes = np.zeros(num_nodes, dtype=bool)
    active_nodes[seed_nodes] = True
    
    # Initialize threshold
    threshold = 0.0

    # Initialize iteration counter
    iteration = 0

    while not np.all(active_nodes) and iteration < max_iterations: # Checks if there are still inactive nodes
        new_active_nodes = np.zeros(num_nodes, dtype=bool) # create zeroes-array to store activated nodes

        # Set threshold for this iteration
        #if iteration == 0:
        #    threshold = 0.0  # Set initial threshold

        # Iterate over each node
        for node_idx in range(num_nodes):
            print(f"Current node: {node_idx+1}") # prints current node
            if not active_nodes[node_idx]:
                neighbors = connectivity_matrix[node_idx] # Retrieves row corresponding to the current node, aka neighbors
                neighbors_activated = active_nodes[np.where(neighbors > 0)[0]]
                node_weighted_degree = np.sum(connectivity_matrix[node_idx] > 0) # weighted degree of node
                weighted_edge = connectivity_matrix[seed_nodes][node_idx]
                if iteration !== 0:
                    weighted_edge += 
                # Calculate activation percentage
                activation_percentage = weighted_edge / node_weighted_degree * 100

                # Update threshold if necessary
                if activation_percentage > threshold:
                    threshold = activation_percentage

                activation_condition = threshold < activation_percentage
                if activation_condition:
                    new_active_nodes[node_idx] = True

        # Update active nodes
        active_nodes = np.logical_or(active_nodes, new_active_nodes)

        iteration += 1
        print(f"Iteration {iteration}: Number of active nodes = {np.sum(active_nodes)}")

    # Check if all nodes are activated
    if np.all(active_nodes):
        print(f"Successful simulation with threshold: {threshold}%")
        successful_thresholds.append(threshold)

# Return the highest threshold allowing simulation completion
if successful_thresholds:
    highest_threshold = max(successful_thresholds)
    print("Highest threshold allowing simulation completion:", highest_threshold)
    return highest_threshold
else:
    print("Unable to find a threshold allowing simulation completion.")
    return None


SyntaxError: invalid syntax (4047190252.py, line 37)

In [ ]:
# simulation function
def simulate_activation(connectivity_matrix, seed_nodes, activation_threshold, max_iterations):
    num_nodes = connectivity_matrix.shape[0]
    #print('num_nodes:', num_nodes)
    #print('seed_nodes:', seed_nodes)
    active_nodes = np.zeros(num_nodes, dtype=bool)
    #print('active_nodes:', active_nodes) 
    # Everything should be false, because created a new array with same num nodes and zeroed out, except for the seeds
    active_nodes[seed_nodes] = True
    #print('active_nodes2:', active_nodes[seed_nodes])
    #print('active_nodes3:', active_nodes)
    
    #active_nodes_history = []
    iteration = 0  # Initialize iteration counter
    
    while not np.all(active_nodes) and iteration < max_iterations: # Checks if there are still inactive nodes. Continues till all nodes are activated
        new_active_nodes = np.zeros(num_nodes, dtype=bool) # Array to store nodes that become activated in current iteration
        # Initially all nodes are inactive
        for node_idx in range(num_nodes): # Iterates over each node in the network
            if not active_nodes[node_idx]: #  Checks if current node is inactive
                neighbors = connectivity_matrix[node_idx] # Retrieves row corresponding to the current node, aka neighbors
                # Selects activated neighbors of current node by filtering active_nodes array based on non-zero entries in neighbors row
                neighbors_activated = active_nodes[np.where(neighbors > 0)[0]]
                degree_of_node = np.sum(connectivity_matrix[node_idx] > 0)  # Degree of the node by counting non-zero entries in row
                # Check if the number of activated neighbors exceeds the threshold
                # Threshold should be highest values that allows completion of cascade
                activation_condition = np.sum(neighbors_activated) / degree_of_node * 100 > activation_threshold
                if activation_condition:
                    # If so, then node becomes activated
                    new_active_nodes[node_idx] = True
        # Updates active_nodes array by performing a logical OR operation with new_active_nodes array 
        # Activated nodes from the current iteration are added to the overall set of activated nodes          
        active_nodes = np.logical_or(active_nodes, new_active_nodes)

        # Append the indices of active nodes to the history list
        #active_node_indices = np.where(active_nodes)[0]
        #active_nodes_history.append(active_node_indices)

        iteration += 1
        print(f"Iteration {iteration}: Number of active nodes = {np.sum(active_nodes)}")
        #save into array

     # Determine the maximum length of active nodes history
    #max_length = max(len(history) for history in active_nodes_history)

    # Pad the shorter arrays with a placeholder value to make them all the same length
    #padded_history = [np.pad(history, (0, max_length - len(history)), constant_values=-1) for history in active_nodes_history]

    # Convert the list of padded active nodes history into a NumPy array and transpose it
    #active_nodes_history_array = np.array(padded_history).T

    print("Simulation complete.")
    return active_nodes #, active_nodes_history_array

In [206]:
# Main function
def main():
    # Load the connectivity matrix
    connectivity_matrixx = matrix
    #connectivity_matrix = sp.csr_matrix(connectivity_matrix)
    
    # Select seed nodes
    num_seeds = 1

    # Find the highest threshold allowing simulation completion
    #threshold_values = np.arange(0, 0.01, 0.0001).tolist()  # Generate threshold values
    #highest_threshold = find_highest_threshold(connectivity_matrix, max_iterations=10)
    #highest_threshold = 0.0064
    for _ in range(1):
        active_nodes = simulation2(connectivity_matrixx, num_seeds, max_iterations=10)
        print("Final activation state with highest threshold:", active_nodes)
    print('For loop complete')

    '''for node in active_nodes:
        print(node)'''
    ### Adjust later
    # Plot the results (Iterations vs. Number of Active Nodes)
    #plt.figure(figsize=(8, 6))
    #plt.plot(range(len(activated_nodes_counts)), activated_nodes_counts, marker='o', color='red')
    #plt.xlabel('Iterations')
    #plt.ylabel('Number of Active Nodes')
    #plt.title('Progress of Activation')
    #plt.grid(True)
    #plt.show()
    

if __name__ == "__main__":
    main()

all_weighted_degrees: [7.66484533 6.90493067 5.53751969 ... 8.26683955 8.66808192 3.00695568]
num_nodes: 1052
seeds: [59]
(1052,)

 ITERATION: 1 



IndexError: tuple index out of range